In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import json
from datetime import datetime

print("=" * 80)
print("EFFICIENTNETB0 TWO-STAGE TRAINING")
print("=" * 80)
print("\nLoading libraries")

print(f"TensorFlow version: {tf.__version__}")


EFFICIENTNETB0 TWO-STAGE TRAINING

Loading libraries
TensorFlow version: 2.20.0


C:\Users\altai\AppData\Roaming\Python\Python312\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:

# ================================================================================
# CONFIGURATION
# ================================================================================
print("\n" + "=" * 80)
print("STEP 1: CONFIGURATION")
print("=" * 80)

# Paths
DATASET_PATH = r"C:\Users\altai\Desktop\DATASET\tomato leaf diseases dataset(augmented)"
TRAIN_PATH = os.path.join(DATASET_PATH, "training")
VAL_PATH = os.path.join(DATASET_PATH, "validation")
MODEL_SAVE_PATH = r"C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5"
HISTORY_PATH = r"C:\Users\altai\Desktop\TLDI_system\training_history_efficientnet_FIXED.json"

# Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 16  # Increased for better gradient estimates
STAGE1_EPOCHS = 20  # Train custom layers only
STAGE2_EPOCHS = 30  # Fine-tune entire model
STAGE1_LR = 0.001  # Higher LR for custom layers
STAGE2_LR = 0.0001  # Lower LR for fine-tuning

print(f"""
Training Configuration:
   • Model: EfficientNetB0 (Two-Stage Training)
   • Image Size: {IMG_SIZE}x{IMG_SIZE}
   • Batch Size: {BATCH_SIZE}
   • Stage 1: {STAGE1_EPOCHS} epochs (custom layers only, LR={STAGE1_LR})
   • Stage 2: {STAGE2_EPOCHS} epochs (fine-tune all, LR={STAGE2_LR})
   • Total Epochs: {STAGE1_EPOCHS + STAGE2_EPOCHS}
""")

# Verify paths
if not os.path.exists(TRAIN_PATH):
    print(f"ERROR: Training folder not found at: {TRAIN_PATH}")
    exit(1)
if not os.path.exists(VAL_PATH):
    print(f"ERROR: Validation folder not found at: {VAL_PATH}")
    exit(1)

print("Dataset paths verified")



STEP 1: CONFIGURATION

Training Configuration:
   • Model: EfficientNetB0 (Two-Stage Training)
   • Image Size: 224x224
   • Batch Size: 16
   • Stage 1: 20 epochs (custom layers only, LR=0.001)
   • Stage 2: 30 epochs (fine-tune all, LR=0.0001)
   • Total Epochs: 50

Dataset paths verified


In [5]:
# ================================================================================
# DATA PREPARATION
# ================================================================================
print("\n" + "=" * 80)
print("STEP 2: DATA PREPARATION")
print("=" * 80)

# Training data with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Validation data (no augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

print("Loading training data...")
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

print("Loading validation data...")
val_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"""
Data loaded:
   • Training samples: {train_generator.samples}
   • Validation samples: {val_generator.samples}
   • Classes: {train_generator.num_classes}
   • Class names: {list(train_generator.class_indices.keys())}
""")



STEP 2: DATA PREPARATION
Loading training data...
Found 20452 images belonging to 11 classes.
Loading validation data...
Found 5488 images belonging to 11 classes.

Data loaded:
   • Training samples: 20452
   • Validation samples: 5488
   • Classes: 11
   • Class names: ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy', 'Unidentified']



In [6]:
# ================================================================================
# MODEL BUILDING
# ================================================================================
print("\n" + "=" * 80)
print("STEP 3: MODEL BUILDING")
print("=" * 80)

tf.keras.backend.clear_session()

print("Building EfficientNetB0 model...")

# Create input
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Load base model without weights first
base_model = EfficientNetB0(
    weights=None,
    include_top=False,
    input_tensor=inputs
)

print(f"EfficientNetB0 architecture created with {len(base_model.layers)} layers")

# Load ImageNet weights
print("Loading ImageNet weights...")
try:
    weights_path = tf.keras.utils.get_file(
        'efficientnetb0_notop.h5',
        'https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5',
        cache_subdir='models',
        file_hash=None  # Skip hash to avoid corruption
    )
    base_model.load_weights(weights_path)
    print("ImageNet weights loaded successfully!")
except Exception as e:
    print(f"Warning: Could not load ImageNet weights: {e}")
    print("Continuing with random initialization")

# Freeze base model for Stage 1
base_model.trainable = False

# Add custom classification layers
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D(name='global_avg_pool')(x)
x = BatchNormalization(name='bn1')(x)
x = Dropout(0.5, name='dropout1')(x)  # Increased dropout
x = Dense(512, activation='relu', name='dense1')(x)
x = BatchNormalization(name='bn2')(x)
x = Dropout(0.4, name='dropout2')(x)
x = Dense(256, activation='relu', name='dense2')(x)
x = BatchNormalization(name='bn3')(x)
x = Dropout(0.3, name='dropout3')(x)
outputs = Dense(train_generator.num_classes, activation='softmax', name='output')(x)

# Create model
model = Model(inputs, outputs, name='EfficientNetB0_TomatoDisease')

print("\nModel Architecture:")
model.summary()

trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
total_params = model.count_params()
print(f"""
Model built successfully!
   • Total parameters: {total_params:,}
   • Trainable parameters: {trainable_params:,}
   • Frozen parameters: {total_params - trainable_params:,}
""")



STEP 3: MODEL BUILDING

Building EfficientNetB0 model...
EfficientNetB0 architecture created with 237 layers
Loading ImageNet weights...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 27s 2us/step
ImageNet weights loaded successfully!

Model Architecture:


Model: "EfficientNetB0_TomatoDisease"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pool                 │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 1280)           │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 512)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 256)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout3 (Dropout)              │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 11)             │         2,827 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,847,790 (18.49 MB)

 Trainable params: 794,123 (3.03 MB)

 Non-trainable params: 4,053,667 (15.46 MB)


Model built successfully!
   • Total parameters: 4,847,790
   • Trainable parameters: 794,123
   • Frozen parameters: 4,053,667



In [12]:
# ================================================================================
# STAGE 1: TRAIN CUSTOM LAYERS ONLY
# ================================================================================
print("\n" + "=" * 80)
print("STAGE 1: TRAINING CUSTOM LAYERS (Base Frozen)")
print("=" * 80)

# Compile for Stage 1
model.compile(
    optimizer=Adam(learning_rate=STAGE1_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for Stage 1
stage1_callbacks = [
    ModelCheckpoint(
        MODEL_SAVE_PATH.replace('.h5', '_stage1.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print(f"""
Starting Stage 1 Training:
   • Epochs: {STAGE1_EPOCHS}
   • Learning Rate: {STAGE1_LR}
   • Base Model: FROZEN
   • Training: Custom layers only
""")

start_time = datetime.now()

history_stage1 = model.fit(
    train_generator,
    epochs=STAGE1_EPOCHS,
    validation_data=val_generator,
    callbacks=stage1_callbacks,
    verbose=1
)

stage1_time = datetime.now() - start_time
print(f"\nStage 1 completed in {stage1_time}")
print(f"Best validation accuracy: {max(history_stage1.history['val_accuracy'])*100:.2f}%")



STAGE 1: TRAINING CUSTOM LAYERS (Base Frozen)

Starting Stage 1 Training:
   • Epochs: 20
   • Learning Rate: 0.001
   • Base Model: FROZEN
   • Training: Custom layers only

Epoch 1/20
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1350 - loss: 2.7577
Epoch 1: val_accuracy improved from None to 0.08710, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED_stage1.h5



Epoch 1: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED_stage1.h5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 1805s 1s/step - accuracy: 0.1865 - loss: 2.4465 - val_accuracy: 0.0871 - val_loss: 3.2043 - learning_rate: 0.0010
Epoch 2/20
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3234 - loss: 1.9264
Epoch 2: val_accuracy improved from 0.08710 to 0.16454, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED_stage1.h5



Epoch 2: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED_stage1.h5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 2402s 2s/step - accuracy: 0.3520 - loss: 1.8408 - val_accuracy: 0.1645 - val_loss: 13.2985 - learning_rate: 0.0010
Epoch 3/20
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4550 - loss: 1.5519
Epoch 3: val_accuracy did not improve from 0.16454
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 2899s 2s/step - accuracy: 0.4835 - loss: 1.4671 - val_accuracy: 0.0909 - val_loss: 17.6712 - learning_rate: 0.0010
Epoch 4/20
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5741 - loss: 1.2215
Epoch 4: val_accuracy improved from 0.16454 to 0.17238, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED_stage1.h5



Epoch 4: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED_stage1.h5

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 1914s 1s/step - accuracy: 0.5882 - loss: 1.1764 - val_accuracy: 0.1724 - val_loss: 4.7175 - learning_rate: 0.0010
Epoch 5/20
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.6403 - loss: 1.0160
Epoch 5: val_accuracy did not improve from 0.17238
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 645s 504ms/step - accuracy: 0.6539 - loss: 0.9839 - val_accuracy: 0.1518 - val_loss: 7.7666 - learning_rate: 5.0000e-04
Epoch 6/20
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.6772 - loss: 0.9214
Epoch 6: val_accuracy did not improve from 0.17238
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 644s 503ms/step - accuracy: 0.6821 - loss: 0.9027 - val_accuracy: 0.1226 - val_loss: 10.5785 - learning_rate: 5.0000e-04
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 

In [13]:
# ================================================================================
# STAGE 2: FINE-TUNE ENTIRE MODEL
# ================================================================================
print("\n" + "=" * 80)
print("STAGE 2: FINE-TUNING ENTIRE MODEL (Base Unfrozen)")
print("=" * 80)

# Unfreeze base model
base_model.trainable = True

# Freeze first 100 layers (keep low-level features frozen)
for layer in base_model.layers[:100]:
    layer.trainable = False

trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
print(f"✓ Base model unfrozen (first 100 layers still frozen)")
print(f"   • Trainable parameters: {trainable_params:,}")

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=STAGE2_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for Stage 2
stage2_callbacks = [
    ModelCheckpoint(
        MODEL_SAVE_PATH,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-8,
        verbose=1
    )
]

print(f"""
Starting Stage 2 Training:
   • Epochs: {STAGE2_EPOCHS}
   • Learning Rate: {STAGE2_LR}
   • Base Model: PARTIALLY UNFROZEN
   • Training: Entire model (fine-tuning)
""")

stage2_start = datetime.now()

history_stage2 = model.fit(
    train_generator,
    epochs=STAGE2_EPOCHS,
    validation_data=val_generator,
    callbacks=stage2_callbacks,
    verbose=1
)

stage2_time = datetime.now() - stage2_start
total_time = datetime.now() - start_time

print(f"\nStage 2 completed in {stage2_time}")
print(f"Best validation accuracy: {max(history_stage2.history['val_accuracy'])*100:.2f}%")


STAGE 2: FINE-TUNING ENTIRE MODEL (Base Unfrozen)
✓ Base model unfrozen (first 100 layers still frozen)
   • Trainable parameters: 4,634,471

Starting Stage 2 Training:
   • Epochs: 30
   • Learning Rate: 0.0001
   • Base Model: PARTIALLY UNFROZEN
   • Training: Entire model (fine-tuning)

Epoch 1/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step - accuracy: 0.3280 - loss: 1.8902
Epoch 1: val_accuracy improved from None to 0.09457, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5



Epoch 1: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 681s 518ms/step - accuracy: 0.3528 - loss: 1.8288 - val_accuracy: 0.0946 - val_loss: 9.1534 - learning_rate: 1.0000e-04
Epoch 2/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step - accuracy: 0.4045 - loss: 1.6730
Epoch 2: val_accuracy did not improve from 0.09457
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 652s 510ms/step - accuracy: 0.4249 - loss: 1.6303 - val_accuracy: 0.0853 - val_loss: 13.6007 - learning_rate: 1.0000e-04
Epoch 3/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.4989 - loss: 1.4399
Epoch 3: val_accuracy improved from 0.09457 to 0.09585, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5



Epoch 3: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 650s 508ms/step - accuracy: 0.5211 - loss: 1.3794 - val_accuracy: 0.0958 - val_loss: 19.2263 - learning_rate: 1.0000e-04
Epoch 4/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step - accuracy: 0.5652 - loss: 1.2457
Epoch 4: val_accuracy improved from 0.09585 to 0.10860, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5



Epoch 4: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5

Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 647s 506ms/step - accuracy: 0.5815 - loss: 1.1979 - val_accuracy: 0.1086 - val_loss: 20.7177 - learning_rate: 1.0000e-04
Epoch 5/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.6449 - loss: 1.0246
Epoch 5: val_accuracy improved from 0.10860 to 0.10969, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5



Epoch 5: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 650s 508ms/step - accuracy: 0.6569 - loss: 0.9916 - val_accuracy: 0.1097 - val_loss: 23.9693 - learning_rate: 5.0000e-05
Epoch 6/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6854 - loss: 0.9195
Epoch 6: val_accuracy did not improve from 0.10969
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 7979s 6s/step - accuracy: 0.6899 - loss: 0.9014 - val_accuracy: 0.0791 - val_loss: 45.0016 - learning_rate: 5.0000e-05
Epoch 7/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.7057 - loss: 0.8553
Epoch 7: val_accuracy improved from 0.10969 to 0.14158, saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5



Epoch 7: finished saving model to C:\Users\altai\Desktop\TLDI_system\trained_model_efficientnet_FIXED.h5

Epoch 7: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 598s 468ms/step - accuracy: 0.7080 - loss: 0.8498 - val_accuracy: 0.1416 - val_loss: 42.1117 - learning_rate: 5.0000e-05
Epoch 8/30
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step - accuracy: 0.7344 - loss: 0.7785
Epoch 8: val_accuracy did not improve from 0.14158
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 596s 466ms/step - accuracy: 0.7346 - loss: 0.7720 - val_accuracy: 0.1091 - val_loss: 48.1775 - learning_rate: 2.5000e-05
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 1.

Stage 2 completed in 3:27:33.468287
Best validation accuracy: 14.16%


In [14]:
# ================================================================================
# FINAL EVALUATION
# ================================================================================
print("\n" + "=" * 80)
print("FINAL EVALUATION")
print("=" * 80)

val_loss, val_accuracy = model.evaluate(val_generator)

print(f"""
📊 FINAL RESULTS:
   ✅ Validation Loss: {val_loss:.4f}
   ✅ Validation Accuracy: {val_accuracy*100:.2f}%
   
   Stage 1 Best: {max(history_stage1.history['val_accuracy'])*100:.2f}%
   Stage 2 Best: {max(history_stage2.history['val_accuracy'])*100:.2f}%
   
   Total Training Time: {total_time}
""")



FINAL EVALUATION
343/343 ━━━━━━━━━━━━━━━━━━━━ 56s 162ms/step - accuracy: 0.0946 - loss: 9.1534

📊 FINAL RESULTS:
   ✅ Validation Loss: 9.1534
   ✅ Validation Accuracy: 9.46%

   Stage 1 Best: 17.24%
   Stage 2 Best: 14.16%

   Total Training Time: 6:23:01.622711

